# Train a sensor processing model using a Convolutional Variational Autoencoder 

Using the Julian-8897-Conv-VAE-PyTorch implementation to train a sensor processing model based on convolutional variational autoencoder. 

The parameters of the training are described by an experiment run of type "sensorprocessing_conv_vae". The result of runing the code in this notebook is the model files that are stored in the experiment directory. 

As the model files will have unpredictable date-time dependent names, after running a satisfactory model, the mode name and directory will need to be copied to the experiment/run yaml file, in the model_subdir and model_checkpoint fields.


In [ ]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
from pprint import pprint
import shutil

# adding the Julian-8897-Conv-VAE-PyTorch into the path
sys.path.append(Config()["conv_vae"]["code_dir"])

# At some point in the development, this hack was necessary for some reason. 
# It seems that as of Feb 2025, the code runs on Windows and Linux without it.
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

from conv_vae import get_conv_vae_config, create_configured_vae_json, train

In [ ]:
# If it is set to true, no actual copying will be done
dry_run = False

# Specify and load the experiment
experiment = "sensorprocessing_conv_vae"
# run = "proprio_128" 
run = "proprio_256" 
exp = Config().get_experiment(experiment, run)
pprint(exp)

### Create the training data for the Conv-VAE

We collect the training data for the Conv-VAE by gathering all the pictures from all the demonstrations of a specific task. One can select the pictures by creating a specific task, and copy there all the relevant demonstrations. 

The collected pictures are put in a newly created training directory for the run:

```
$experiment\vae-training-data\Images\*.jpg
```

In [ ]:
def copy_images_to_training_dir(taskname, training_image_dir):
    """Copy all the images from a specific task into the training image dir."""
    task_dir = pathlib.Path(demos_dir, taskname)
    # _, task_dir = ui_choose_task(offer_task_creation=True)

    for demo in task_dir.iterdir():
        if not demo.is_dir(): continue
        for item in demo.iterdir():
            if item.suffix != ".jpg": continue
            name = f"{demo.name}_{item.stem}.jpg"
            destination = pathlib.Path(training_image_dir, name)
            print(f"copy {item} to \n{destination}")
            if not dry_run:
                shutil.copyfile(item, destination)

In [ ]:
demos_top = pathlib.Path(Config()["demos"]["directory"])
demos_dir = pathlib.Path(demos_top, "demos")

subdir_count = sum(1 for item in demos_dir.iterdir() if item.is_dir())
print(f"Number of demo directories: {subdir_count}")

# Deciding on the location of the training data
training_data_dir = pathlib.Path(exp["data_dir"], exp["training_data_dir"])
# training_data_dir = pathlib.Path(Config()["conv_vae"]["training_data_dir"])
training_image_dir = pathlib.Path(training_data_dir, "Images")
training_image_dir.mkdir(exist_ok = False, parents=True)

print(f"Training data dir={training_image_dir}")

# Define a set of common image file extensions
image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp"}
# Count the image files
image_count = sum(1 for item in training_image_dir.iterdir() if item.suffix.lower() in image_extensions and item.is_file())

print(f"Number of image files in training dir: {image_count}")

if image_count == 0:
    taskname = exp['training_task']
    copy_images_to_training_dir(
        taskname = taskname, training_image_dir=training_image_dir)
else:
    print("There are already images in training image dir {training_image_dir}. Do not repeat the copying.")            


# Run the training

Actually run the training. This is done by creating the json-based configuration file of the Conv-VAE library with the parameters specified in the library. Then we call the code of the library to perform the training. 

In [ ]:
# Create the vae configuration, based on the experiment
file = create_configured_vae_json(exp)
print(file)
vae_config = get_conv_vae_config(file)

# actually run the training
print(f'Running the trainer from scratch for {vae_config["trainer"]["epochs"]}')
trainer = train(vae_config)

In [ ]:
# These are the metrics recorded
# they are of utils/util.py / MetricTracker which has a pandas dataframe as data
print(trainer.train_metrics)
print(trainer.valid_metrics)
# 
trainer.train_metrics._data
# trainer.valid_metrics._data

__Important__ After the training finished, in order to use the resulting system, one need to edit the run file (eg: vae_01.yaml) and enter into it the location of the checkpoint. This is the content printed by the code cell below

In [ ]:

print(f"model_subdir: '{trainer.checkpoint_dir.name}'")
print(f"model_checkpoint: 'checkpoint-epoch{trainer.epochs}.pth'")


In [ ]:
if "exp_run_sys_dep_file" in exp:
    print(f'The text above to be put into \n the system dependent experiment run file {exp["exp_run_sys_dep_file"]}')
else:
    print(f'As the system dependent experiment run file does not exist,\n the text can be put into the system independent file\n {exp["exp_run_sys_indep_file"]}')
